# 1. 단일 이미지 추론
- `detect.py`를 사용할 경우 [YOLOv5](https://github.com/ultralytics/yolov5)를 clone 받고 해당 레포지토리에서 실행

In [1]:
import torch
from PIL import Image
from io import BytesIO

# GPU가 사용 가능한 경우 cuda 사용 / GPU가 사용 불가능한 경우 CPU로 초기화하여 CPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

device: cuda


In [2]:
## 전처리 함수
def preprocess_image(image, image_size, crop_size, interpolation=Image.LANCZOS):
    '''
    Parameters
    - image: (PIL.JpegImagePlugin.JpegImageFile) 이미지 파일
    - image_size: (int) yolo input 이미지 사이즈
    - crop_size: (list, tuple) "좌-상-우-하" 순서로 자를 길이
    - interpolation: (int) interpolation 방식 / default: Image.LANCZOS
    '''

    # 이미지 crop -> 정사각형으로 만들기
    image_width, image_height = image.size
    cropped = image.crop((crop_size[0], crop_size[1], image_width-crop_size[2], image_height-crop_size[3]))

    # resize
    resized = cropped.resize((image_size, image_size), interpolation)
    b = BytesIO()
    resized.save(b, 'jpeg')
    return Image.open(b)

In [42]:
# 이미지 이름
image_name = 'wheel.jpg'

# 이미지 load
image = Image.open('folder_name/' + image_name)

# 전처리 수행
image = preprocess_image(image, 2048, (200, 0, 200, 0))

In [46]:
## 모델 ##

model = torch.hub.load('.', 'custom','weights/best_0511.pt', source='local')

# confidence를 높여서 더욱 정밀하게 추론하도록 만듦
model.conf = 0.7

# 하나의 bbox에 대해 여러 라벨을 할당하지 못하도록 만듦
model.multi_label = False

YOLOv5  v7.0-163-g016e046 Python-3.9.13 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)



requirements: C:\Users\SSAFY\dev\pjt3\requirements.txt not found, check failed.


Fusing layers... 
Model summary: 346 layers, 76126356 parameters, 0 gradients, 109.9 GFLOPs
Adding AutoShape... 


In [47]:
## 추론 ##
results = model(image, size=2048)

In [48]:
## 추론 결과
# results.show()              # 추론 이미지 보기
# results.save()              # 추론 이미지 저장

# bbox x좌측좌표/y위쪽좌표/x오른쪽좌표/y아래좌표 데이터 저장
bolts = results.xyxy[0].tolist()

# bbox x중앙좌표/y중앙좌표/가로/세로 데이터(normalized) 저장
bolts_norm = results.xywhn[0].tolist()

Saved 1 image to runs\detect\exp10


In [20]:
# 확장자 제거
image_name_revised = image_name.replace('.jpg', '')

# 볼트 이미지 저장 폴더
bolt_folder = 'bolt_images/'

# 볼트 좌표를 저장할 텍스트 파일 열기
with open('{} + {}'.format(bolt_folder, image_name_revised) + '.txt', 'w') as f:
    for idx, bolt in enumerate(bolts):
        # x좌측좌표/y위쪽좌표/x오른쪽좌표/y아래좌표
        x_min, y_min, x_max, y_max = list(map((lambda x: x.item()), bolt[:4]))
        
        # 볼트 이미지 crop
        cropped = image.crop((x_min, y_min, x_max, y_max))
        
        # 볼트 이미지 저장
        cropped.save(bolt_folder + '{}_{}.jpg'.format(image_name_revised, idx+1))

        # yolo 라벨(label) 형식에 맞게 텍스트 작성
        bbox_str = '{} {} {} {} {}'.format(int(bolts_norm[idx][-1]), bolts_norm[idx][0], bolts_norm[idx][1], bolts_norm[idx][2], bolts_norm[idx][3])
        f.write(bbox_str + '\n')
    f.close()

In [52]:
## detect.py 파일로 추론 수행 ##

# # Hyperparameters
# image_size = 2048                       # 이미지 사이즈
# image_path = 'wheel.jpg'                # 추론 대상 이미지
# data_path = 'data/wheel.yaml'           # data 파일
# weights_path = 'weights/yolo_0511.pt'   # weight 파일
# save_path = 'results/test'              # 추론 결과 저장 폴더
# conf_thres = 0.89                       # confidence threshold
# device = torch.device("cuda:0")         # 추론에 사용할 device

# !python detect.py --weights {weights_path} --img {image_size} --conf {conf_thres} --source {image_path} --project {save_path} --device {device} --save-txt

requirements: C:\Users\SSAFY\dev\pjt3\requirements.txt not found, check failed.


detect: weights=['weights/best_0511.pt'], source=datasets/test_presentation/lost_0508_27.jpg, data=data\coco128.yaml, imgsz=[2048, 2048], conf_thres=0.89, iou_thres=0.45, max_det=1000, device=cuda:0, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=results/test_0517, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-163-g016e046 Python-3.9.13 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)

Fusing layers... 
Model summary: 206 layers, 12312052 parameters, 0 gradients, 16.1 GFLOPs
image 1/1 C:\Users\SSAFY\dev\pjt3\yolov5\datasets\test_presentation\lost_0508_27.jpg: 1728x2048 3 inners, 8 outers, 104.4ms
Speed: 3.7ms pre-process, 104.4ms inference, 67.8ms NMS per image at shape (1, 3, 2048, 2048)
Results saved to results\test_0517\exp
1 labels saved to results\test_0517\exp\labels


# 2. 여러 이미지 추론

In [35]:
## CONSTANTS ##

MODEL_PATH = 'weights/'                             # weights 폴더
MODEL_NAME = 'yolo_0511.pt'                         # weights 파일 이름
IMAGE_PATH = 'images/'                              # 추론 대상 이미지들이 있는 폴더
IMAGE_SIZE = 2048                                   # 이미지 사이즈
CROP_SIZE = (200, 0, 200, 0)                        # 전처리 시 수행할 크롭 사이즈(좌, 상, 우, 하)
BOLT_IMAGE_PATH = 'bolt_images/'                    # 볼트 이미지를 저장할 폴더
CONFIDENCE = 0.95                                   # confidence threshold
MULTI_LABEL = False                                 # multi label 비허용

In [36]:
import torch
import os
from PIL import Image
from io import BytesIO

## 모델
model = torch.hub.load('.', 'custom',MODEL_PATH + MODEL_NAME, source='local')

# confidence를 높여서 더욱 정밀하게 추론하도록 만듦
model.conf = CONFIDENCE

# 하나의 bbox에 대해 여러 라벨을 할당하지 못하도록 만듦
model.multi_label = MULTI_LABEL


## 전처리 함수
def preprocess_image(image, image_size, crop_size, interpolation=Image.LANCZOS):
    '''
    Parameters
    - image: (PIL.JpegImagePlugin.JpegImageFile) 이미지 파일
    - image_size: (int) yolo input 이미지 사이즈
    - crop_size: (list, tuple) "좌-상-우-하" 순서로 자를 길이
    - interpolation: (int) interpolation 방식 / default: Image.LANCZOS
    '''

    # 이미지 crop -> 정사각형으로 만들기
    image_width, image_height = image.size
    cropped = image.crop((crop_size[0], crop_size[1], image_width-crop_size[2], image_height-crop_size[3]))

    # resize
    resized = cropped.resize((image_size, image_size), interpolation)
    b = BytesIO()
    resized.save(b, 'jpeg')
    return Image.open(b)

YOLOv5  v7.0-163-g016e046 Python-3.9.13 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)



requirements: C:\Users\SSAFY\dev\pjt3\requirements.txt not found, check failed.


Fusing layers... 
Model summary: 206 layers, 12312052 parameters, 0 gradients, 16.1 GFLOPs
Adding AutoShape... 


In [38]:
## 추론할 이미지 정의 ##

# 이미지 폴더에는 반드시 이미지만 넣을 것
image_folder = IMAGE_PATH
image_list = os.listdir(image_folder)

if '.ipynb_checkpoints' in image_list:
    image_list.remove('.ipynb_checkpoints')

# print(len(image_list))

101


In [39]:
for image_name in image_list:
    image = Image.open(image_folder + image_name)
    image = preprocess_image(image, IMAGE_SIZE, CROP_SIZE)

    results = model(image, size=IMAGE_SIZE)
    bolts = results.xyxy[0].tolist()
    bolts_norm = results.xywhn[0].tolist()

    image_name_revised = image_name.replace('.jpg', '')
    with open(BOLT_IMAGE_PATH + '{}.txt'.format(image_name_revised), 'w') as f:
        for idx, bolt in enumerate(bolts):
            x_min, y_min, x_max, y_max = list(map((lambda x: x.item()), bolt[:4]))
            cropped = image.crop((x_min, y_min, x_max, y_max))
            cropped.save(BOLT_IMAGE_PATH + '{}_{}.jpg'.format(image_name_revised, idx+1))

            bbox_str = '{} {} {} {} {}'.format(int(bolts_norm[idx][-1]), bolts_norm[idx][0], bolts_norm[idx][1], bolts_norm[idx][2], bolts_norm[idx][3])
            f.write(bbox_str + '\n')
        f.close()